In [ ]:
import os
os.chdir('..')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from sklearn.preprocessing import StandardScaler
np.random.seed(3)

In [ ]:
N = 20000
t_max = 100
T_comp = 10
T_sig = 1
t = np.linspace(1, t_max, N)

## Components

In [ ]:
def generate_rectangular_signal_with_random_rectangle_lenght(min_lengh:int, max_lenght:int, length: int):
    zeros = np.random.randint(min_lengh, max_lenght, length)
    ones = np.random.randint(min_lengh, max_lenght, length)
    ls = []
    for l in [[1]*lh+[-1]*ll for lh, ll in zip(zeros, ones)]:
        ls.extend(l)
    if len(ls) > length:
        return np.array(ls[:length])
    else:
        print('Not enouhg samples, run again')

In [ ]:
sequence_a = generate_rectangular_signal_with_random_rectangle_lenght(500, 1500, N)
sequence_b = generate_rectangular_signal_with_random_rectangle_lenght(500, 1500, N)
sequence_c = generate_rectangular_signal_with_random_rectangle_lenght(500, 1500, N)

plt.plot(sequence_a[0:5000])
plt.plot(sequence_b[0:5000])
plt.plot(sequence_c[0:5000])

# Signals comp a

# What do i want to show?
1. The model fit of a holistic autoencoder-predictor is better than the one which is fittet on a single components data. This is due to the fact, that one component has an influence on the other.
    - simulate three signals representing whether the component is active or not (causal factor of variation in the data)
    - simulate one or two signals for each components, relying on all the component signals with time lag.
    - Train two models:
        - For the somponents signals only
        - For all the components
2. The symptom can be localized better if we train individual decoder-predictors, than if we use the holistic decoder.
   - simulate a failure (e.g. one of the signals does not turn on even though the component is switched on.
   - Show, that the model fit of the individual models is capable of localizing the fault, wheras the holistic model is not.


In [ ]:
LAG = 100

In [ ]:
sig1_init = 0
sig1_v_init = 0

In [ ]:
x = 1/2 a t**2 + v t

In [ ]:
a = [1]*10+[0]*10
a = [*a]*5
len(a)

In [ ]:
sig_1 = [sig1_init]
sig_1_v = [sig1_v_init]
# a = [sequence_a]
a = [1]*10+[0]*10
a = [*a]*5
for i in range(1, 10000):
    sig_1_v.append(a[i-1] + sig_1_v[i-1])
    sig_1.append(sig_1_v[i-1]+sig_1[i-1])







# for i in range(1, len(sequence_a)):
#     sig_1.append(sig_1[i-1] + sig_1_v[i-1] * (sequence_a[i-1]-sig_1[i-1]))
    
    
    
               

In [ ]:
plt.plot(sig_1)

In [ ]:
plt.plot(sig_1[0:1000])
plt.plot(sequence_a[0:1000])

In [ ]:
sig_1_init = 1
sign_1_m = 1/100
sig_1 = [sig_1_init]
# [sig_1[ind-1]+sign_1_m if sequence_a[ind]==1 else sig_1[ind-1]+sign_1_m for ind in range(1,N-100)] 

In [ ]:
[sig_1[ind-1] for ind in range(1,2)]

In [ ]:
list(range(1,N))

In [ ]:
sig_1 =  \
    + sequence_a *np.sin(2* np.pi * 1/(T_sig) * t) \
    + .2 *np.random.rand(N) \
    # + sig_b[:-LAG]
# sig_2 =  \
#     + sequence_a * signal.square(2*np.pi * 1/(T_sig) * t) \
#     +  .2 *np.random.rand(N) \
# #     - sig_b[:-LAG]

# sig_3 =  \
#     + sequence_a *signal.sawtooth(2* np.pi * 1/(T_sig) * t) \
#     + .2 *np.random.rand(N) \
#     # - sig_b[:-LAG]



plt.plot(sequence_a[0:10000])
plt.plot(sig_1[0:10000])


In [ ]:
plt.plot(sequence_a[0:10000])
plt.plot(sig_2[0:10000])

In [ ]:
plt.plot(sequence_a[0:10000])
plt.plot(sig_3[0:10000])

In [ ]:
df = pd.DataFrame(dict(sig_1=sequence_a,
                       sig_2=sequence_b,
                       sig_3=sequence_c,
                       # sig_4=sig_4,
                       # sig_5=sig_5,
                       # sig_6=sig_6,
                       # sig_7=sig_7,
                      ))
TRAIN_DATA_PATH = 'data/trainin_data.csv'
df.to_csv(TRAIN_DATA_PATH, index=False)

In [ ]:
df.head()

## test dataset

In [ ]:
from diag_vae.datamodule import DiagTsDataset

In [ ]:
ds = DiagTsDataset(data_path='data/trainin_data.csv', seq_len=1000, cols=[f'sig_{i+1}' for i in range(3)])

In [ ]:
ds.__getitem__(2)[0].shape

In [ ]:
plt.plot(ds.__getitem__(2)[0].numpy().T)

In [ ]:
plt.plot(ds.__getitem__(2)[1].numpy().T)

## test data module

In [ ]:
from diag_vae.datamodule import DiagTsDataModule

In [ ]:
dm = DiagTsDataModule(data_path='data/trainin_data.csv', seq_len=1000, cols=[f'sig_{i+1}' for i in range(3)])
sample_x, sample_y = next(iter(dm.train_dataloader()))

In [ ]:
index = 5
plt.plot(sample_x[index, :, :].numpy().T)

In [ ]:
index = 5
plt.plot(sample_y[index, :, :].numpy().T)

## check model output

In [ ]:
from diag_vae.ae import Conv1dAE


In [ ]:
loggs_dir = 'loggs/tcn_ae/version_21/checkpoints'
checkpoint = os.path.join(loggs_dir, f'{os.listdir(loggs_dir)[0]}')
checkpoint
model = Conv1dAE.load_from_checkpoint(checkpoint)

In [ ]:
sample_z = model.encode(sample_x)
sample_x_hat, sample_y_hat = model.decode(sample_z)

In [ ]:
plt.plot(sample_x[index, :, :].detach().numpy().T)

In [ ]:
plt.plot(sample_x_hat[index, :, :].detach().numpy().T)

In [ ]:
plt.plot(sample_y_hat[index, :, :].detach().numpy().T)